In [1]:
# Load in the packages
import numpy as np
import folktables
from folktables import ACSDataSource, ACSIncome

import os
import sys
import copy
sys.path.append('..')
import random
import FairCertModule
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import models, transforms
from FullyConnected import FullyConnected
import pytorch_lightning as pl

# Set random seeds
SEED = 0
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

dataset = "Coverage"
if(dataset == "Employ"):
    TEST_EPSILON = 0.075
if(dataset == "Coverage"):
    TEST_EPSILON = 0.25
else:
    TEST_EPSILON = 0.05   


In [2]:
# Data loaders
import folk_utils
X_train, X_test, X_val, y_train, y_test, y_val, lp_epsilon, sr_epsilon, g_train, g_test, g_val = folk_utils.get_dataset(dataset, groups=True)
f_epsilon = lp_epsilon

/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


In [3]:
print(len(g_train))
print(sum(g_train))

108058
59582.0


In [4]:
# Load in the trained models

MODEL_STATE  = 'CA'
MODEL_YEAR   = '2015'
MODEL_WIDTH  = '256'
MODEL_METRIC = 'LP'

sgd_id = "SGD" 
pgd_id = "FAIR-PGD" 
ibp_id = "FAIR-IBP" 
glob_id = "FAIR-DRO"
ibpg_id = "FAIR-IBPG"

def load_model_from_id(model_id, dataset, width=MODEL_WIDTH):
    model = FullyConnected(hidden_lay=2, hidden_dim=256, dataset=dataset)
    ckpt = torch.load("%sModels/%s.ckpt"%(dataset, model_id))
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    checkpoint = torch.load("%sModels/%s.ckpt"%(dataset, model_id))
    model.load_state_dict(torch.load('%sModels/%s.pt'%(dataset, model_id)))
    return model

sgd_model = load_model_from_id(sgd_id, dataset)
pgd_model = load_model_from_id(pgd_id, dataset)
ibp_model = load_model_from_id(ibp_id, dataset)
glob_model = load_model_from_id(glob_id, dataset)
ibpg_model = load_model_from_id(ibpg_id, dataset)


In [5]:
from folk_utils import evaluate_accuracy, evaluate_delta_PGD, evaluate_delta_IBP

In [6]:
sgd_acc = evaluate_accuracy(sgd_model, X_test, y_test)
pgd_acc = evaluate_accuracy(pgd_model, X_test, y_test)
ibp_acc = evaluate_accuracy(ibp_model, X_test, y_test)
glob_acc = evaluate_accuracy(glob_model, X_test, y_test) 
ibpg_acc = evaluate_accuracy(ibpg_model, X_test, y_test)

In [7]:
print(sgd_acc)
print(pgd_acc)
print(ibp_acc)
print(glob_acc)
print(ibpg_acc)

tensor(0.7213)
tensor(0.7090)
tensor(0.6994)
tensor(0.6993)
tensor(0.6810)


In [8]:
# Evaluate equalized accuracy
majority = np.squeeze(np.argwhere(g_test == 1))
minority = np.squeeze(np.argwhere(g_test == 0))

print(X_test[np.squeeze(majority)].shape)
sgd_acc_maj = evaluate_accuracy(sgd_model, X_test[majority], y_test[majority])
pgd_acc_maj = evaluate_accuracy(pgd_model, X_test[majority], y_test[majority])
ibp_acc_maj = evaluate_accuracy(ibp_model, X_test[majority], y_test[majority])
glob_acc_maj = evaluate_accuracy(glob_model, X_test[majority], y_test[majority])
ibpg_acc_maj = evaluate_accuracy(ibpg_model, X_test[majority], y_test[majority])

sgd_acc_min = evaluate_accuracy(sgd_model, X_test[minority], y_test[minority])
pgd_acc_min = evaluate_accuracy(pgd_model, X_test[minority], y_test[minority])
ibp_acc_min = evaluate_accuracy(ibp_model, X_test[minority], y_test[minority])
glob_acc_min = evaluate_accuracy(glob_model, X_test[minority], y_test[minority])
ibpg_acc_min = evaluate_accuracy(ibpg_model, X_test[minority], y_test[minority])

print(abs(sgd_acc_maj - sgd_acc_min))
print(abs(pgd_acc_maj - pgd_acc_maj))
print(abs(ibp_acc_maj - ibp_acc_maj))
print(abs(glob_acc_maj - glob_acc_maj))
print(abs(ibpg_acc_maj - ibpg_acc_maj))


torch.Size([16471, 44])
tensor(0.0009)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)


In [9]:
# Demo-Parity Computation
def demographic_parity(model, X, y, g):
    g = torch.Tensor(g).type(torch.LongTensor)
    g = torch.clip(g, 0, 1)
    y_hat = torch.nn.functional.softmax(model(X))
    y_hat = torch.squeeze(y_hat)
    y_maj = torch.squeeze(y_hat[torch.squeeze(torch.argwhere(g == 0))])
    y_min = torch.squeeze(y_hat[torch.squeeze(torch.argwhere(g == 1))])
    total_rate = torch.sum(torch.argmax(y_hat, axis=1))/len(torch.argmax(y_hat, axis=1))
    maj_rate   = torch.sum(torch.argmax(y_maj, axis=1))/len(torch.argmax(y_maj, axis=1))
    min_rate   = torch.sum(torch.argmax(y_min, axis=1))/len(torch.argmax(y_min, axis=1))
    #abs(total_rate - maj_rate) + abs(total_rate - min_rate)
    return float(abs(maj_rate - min_rate).detach().numpy()) 

 
def equalized_odds(model, X, y, g):
    g = torch.Tensor(g).type(torch.LongTensor)
    y_hat = torch.nn.functional.softmax(model(X))
    y_hat = torch.squeeze(y_hat)
    y_true_maj = torch.squeeze(y[torch.squeeze(torch.argwhere(g == 0))])
    y_true_min = torch.squeeze(y[torch.squeeze(torch.argwhere(g == 1))])
    y_maj = torch.squeeze(y_hat[torch.squeeze(torch.argwhere(g == 0))])
    y_min = torch.squeeze(y_hat[torch.squeeze(torch.argwhere(g == 1))])
    temp = y_min[torch.squeeze(torch.argwhere(y_true_min == 1))]
    pos_min_rate = torch.sum(torch.argmax(temp, axis=1))/len(torch.argmax(temp, axis=1))
    temp = y_min[torch.squeeze(torch.argwhere(y_true_min == 0))]
    neg_min_rate = torch.sum(1-torch.argmax(temp, axis=1))/len(torch.argmax(temp, axis=1)) 
    temp = y_maj[torch.squeeze(torch.argwhere(y_true_maj == 1))]
    pos_maj_rate = torch.sum(torch.argmax(temp, axis=1))/len(torch.argmax(temp, axis=1))
    temp = y_maj[torch.squeeze(torch.argwhere(y_true_maj == 0))]
    neg_maj_rate = torch.sum(1-torch.argmax(temp, axis=1))/len(torch.argmax(temp, axis=1))
    return float(abs(pos_min_rate - pos_maj_rate) + abs(neg_min_rate - neg_maj_rate))

# Eq Opp Computation
def equalized_opportunity(model, X, y, g):
    g = torch.Tensor(g).type(torch.LongTensor)
    y_hat = torch.nn.functional.softmax(model(X))
    y_hat = torch.squeeze(y_hat)
    y_true_maj = torch.squeeze(y[torch.squeeze(torch.argwhere(g == 0))])
    y_true_min = torch.squeeze(y[torch.squeeze(torch.argwhere(g == 1))])
    y_maj = torch.squeeze(y_hat[torch.squeeze(torch.argwhere(g == 0))])
    y_min = torch.squeeze(y_hat[torch.squeeze(torch.argwhere(g == 1))])
    temp = y_min[torch.squeeze(torch.argwhere(y_true_min == 1))]
    pos_min_rate = torch.sum(torch.argmax(temp, axis=1))/len(torch.argmax(temp, axis=1))

    temp = y_maj[torch.squeeze(torch.argwhere(y_true_maj == 1))]
    pos_maj_rate = torch.sum(torch.argmax(temp, axis=1))/len(torch.argmax(temp, axis=1))
    
    return float(abs(pos_min_rate - pos_maj_rate))

In [10]:
# Evaluate demographic parity

sgd_dp = demographic_parity(sgd_model, X_test, y_test, g_test)
pgd_dp = demographic_parity(pgd_model, X_test, y_test, g_test)
ibp_dp = demographic_parity(ibp_model, X_test, y_test, g_test)
glob_dp = demographic_parity(glob_model, X_test, y_test, g_test)
ibpg_dp = demographic_parity(ibpg_model, X_test, y_test, g_test)

print(sgd_dp)
print(pgd_dp)
print(ibp_dp)
print(glob_dp)
print(ibpg_dp)

0.006408393383026123
0.05595281720161438
0.033047959208488464
0.03253581374883652
0.028308406472206116


/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


In [11]:
# Evaluate demographic parity

sgd_eo = equalized_odds(sgd_model, X_test, y_test, g_test)
pgd_eo = equalized_odds(pgd_model, X_test, y_test, g_test)
ibp_eo = equalized_odds(ibp_model, X_test, y_test, g_test)
glob_eo = equalized_odds(glob_model, X_test, y_test, g_test)
ibpg_eo = equalized_odds(ibpg_model, X_test, y_test, g_test)

print(sgd_eo)
print(pgd_eo)
print(ibp_eo)
print(glob_eo)
print(ibpg_eo)


/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.023339271545410156
0.11253297328948975
0.07027119398117065
0.06945055723190308
0.0648534744977951


In [12]:
# Evaluate equalized opp

sgd_eo = equalized_opportunity(sgd_model, X_test, y_test, g_test)
pgd_eo = equalized_opportunity(pgd_model, X_test, y_test, g_test)
ibp_eo = equalized_opportunity(ibp_model, X_test, y_test, g_test)
glob_eo = equalized_opportunity(glob_model, X_test, y_test, g_test)
ibpg_eo = equalized_opportunity(ibpg_model, X_test, y_test, g_test)

print(sgd_eo)
print(pgd_eo)
print(ibp_eo)
print(glob_eo)
print(ibpg_eo)


0.008268654346466064
0.0748489499092102
0.055429697036743164
0.054843127727508545
0.05468307435512543


/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [13]:
from folk_utils import evaluate_accuracy, evaluate_delta_PGD, evaluate_delta_IBP
f_epsilon = torch.Tensor(f_epsilon)

def differential_IF(model, X, y, g):
    majority = np.squeeze(np.argwhere(g == 1))
    minority = np.squeeze(np.argwhere(g == 0))
    maj_delta = evaluate_delta_IBP(model, X[majority], y[majority], f_epsilon, TEST_EPSILON, 2)
    min_delta = evaluate_delta_IBP(model, X[minority], y[minority], f_epsilon, TEST_EPSILON, 2)
    return abs(maj_delta - min_delta)

sgd_eo = differential_IF(sgd_model, X_test, y_test, g_test)
pgd_eo = differential_IF(pgd_model, X_test, y_test, g_test)
ibp_eo = differential_IF(ibp_model, X_test, y_test, g_test)
glob_eo = differential_IF(glob_model, X_test, y_test, g_test)
ibpg_eo = differential_IF(ibpg_model, X_test, y_test, g_test)

print(sgd_eo)
print(pgd_eo)
print(ibp_eo)
print(glob_eo)
print(ibpg_eo)

0.0
0.009640098
0.008532524
0.0066285953
0.0008978583
